In [ ]:
import numpy as np
from multiple_stabilizers import get_condition_indices, generate_parity_check_matrix


H, L = 8, 17
m = 3
stabilizer_shape = np.array([[0, 1, 0],
                            [0, 1, 0],
                            [1, 0, 1]])

condition_indices = get_condition_indices(stabilizer_shape)
print("Condition indices (dx, dy):", condition_indices)

H_matrix = generate_parity_check_matrix(H, L, m, condition_indices)

print("Generated Parity-Check Matrix H:")
# print H index of 1
# for i in range(H_matrix.shape[0]):
#     print(f'\n${i} parity-check: ', end='')
#     for j in range(H_matrix.shape[1]):
#         if H_matrix[i][j] == 1:
#             print(f"({i}, {j})", end=' ')
print("\nShape of H matrix:", H_matrix.shape)
# np.set_printoptions(threshold=np.inf, linewidth=np.inf)
# print(H_matrix)

Stabilizer shape:
[[0 1 0]
 [0 1 0]
 [1 0 1]]
Rows: 3, Cols: 3
Condition indices (dx, dy): [(1, 0), (2, -1), (2, 1)]
Generating H matrix of size 102 x 136
Generated Parity-Check Matrix H:

Shape of H matrix: (102, 136)


In [23]:
import numpy as np
from ldpc import bposd_decoder

bpd=bposd_decoder(
    H_matrix,#the parity check matrix
    error_rate=0.05,
    channel_probs=[None], #assign error_rate to each qubit. This will override "error_rate" input variable
    # max_iter=surface_code.N, #the maximum number of iterations for BP)
    bp_method="ms",
    ms_scaling_factor=0, #min sum scaling factor. If set to zero the variable scaling factor method is used
    osd_method="osd_cs", #the OSD method. Choose from:  1) "osd_e", "osd_cs", "osd0"
    osd_order=7 #the osd search depth
    )

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/ldpc/_legacy_ldpc_v1/_legacy_bposd_decoder.py:45: UserWarning: This is the old syntax for the `bposd_decoder` from `ldpc v1`. Use the `BpOsdDecoder` class from `ldpc v2` for additional features.
  warnings.warn(


In [80]:
from multiple_stabilizers import fill_Z_with_stabilizer_shape 
# input_row = np.random.randint(0, 2, size=(m-1, L))
input_row = np.zeros((m-1, L), dtype=int)
input_row[1][8] = 1
print("Input row:")
print(input_row)

Z = fill_Z_with_stabilizer_shape(input_row, H, L, m, [condition_indices], same_shape=True)
print("Generated Z matrix:")
print(Z)
print("Shape of Z matrix:", Z.shape)

Z = Z.flatten()[::-1]  # flatten and reverse
print("Flattened Z vector:")
print(Z)

error = np.zeros_like(Z, dtype=int)
error_indices = np.random.choice(np.arange(len(Z)), size=19, replace=False)
error[error_indices] = 1
print("Error vector:")
print(error)

received = (Z + error) % 2
print("Received vector:")
print(received)

# syndrome calculation
syndrome = np.mod(H_matrix @ received, 2)
print("Syndrome:")
print(syndrome)
print("Shape of syndrome:", syndrome.shape)

bpd.decode(syndrome)

print("Error")
print(error)
print("BP+OSD Decoding")
print(bpd.osdw_decoding)
#Decoding is successful if the residual error commutes with the logical operators
residual_error=(bpd.osdw_decoding + error) %2
print("Residual Error:")
print(residual_error)
a=(Z @ residual_error%2).any()
if a: a="Yes"
else: a="No"
print(f"Logical Error: {a}\n")


Input row:
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]]
condition offsets list: [[(1, 0), (2, -1), (2, 1)]]
Generated Z matrix:
[[0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 1 1 0 0 0 1 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0]]
Shape of Z matrix: (8, 17)
Flattened Z vector:
[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0
 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0]
Error vector:
[0 1 0 0 0 0 0 0 0 0 1 0 1 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 

In [79]:
num_trials = 10000
logical_errors = 0

for _ in range(num_trials):
    # generate random error
    error = np.zeros_like(Z, dtype=int)
    error_indices = np.random.choice(np.arange(len(Z)), size=19, replace=False)
    error[error_indices] = 1

    received = (Z + error) % 2
    syndrome = np.mod(H_matrix @ received, 2)

    bpd.decode(syndrome)
    residual_error = (bpd.osdw_decoding + error) % 2
    logical = (Z @ residual_error % 2).any()

    if logical:
        logical_errors += 1

logical_error_rate = logical_errors / num_trials
print("Logical Error Rate:", logical_error_rate)


Logical Error Rate: 0.0156
